<a href="https://colab.research.google.com/github/nish700/LLM/blob/main/fine_tune_BERT_on_JIGSAW_Multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing transformers
%%capture
!pip install transformers==4.28.0

In [ ]:
import pandas as pd
import transformers

In [ ]:
# loading the drive for the data
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile(file_path,'r') as zsobj:
  zsobj.extractall('./tmp/kaggle')
zsobj.close()

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.shape

(223549, 8)

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
df.toxic.value_counts()

0    202165
1     21384
Name: toxic, dtype: int64

In [ ]:
filter_data = df[['comment_text','toxic']]

In [ ]:
# filter_data = filter_data[:20000]

In [ ]:
filter_data.shape

(223549, 2)

In [ ]:
filter_data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from transformers import TrainingArguments , Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ['I am working','We are dancing']
tokenizer(sample_data, padding=True, truncation=True, max_length = 512)

{'input_ids': [[101, 1045, 2572, 2551, 102], [101, 2057, 2024, 5613, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(filter_data['comment_text'])
y = list(filter_data['toxic'])

#split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
#tokenize
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length = 512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length = 512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
len(X_train) , len(X_test)

(178839, 44710)

In [ ]:
# create torch dataset
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
test_dataset = Dataset(X_test_tokenized, y_test)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,  1000,  1006,  2043,  2017,  6069,  1007,  2507,  2009,  2039,
          2000,  2033,  1996, 10324, 12472,  2069,  6611,  2000,  1996, 10324,
          1997,  4790,  1012,  1996,  3720,  2171,  2001,  2187,  1996,  2126,
          2017,  2359,  2009,  1024,  1000,  1000,  1006,  2043,  2017,  6069,
          1007,  2507,  2009,  2039,  2000,  2033,  1000,  1000,  1012,  3568,
          1010,  1045,  2572,  2025, 20084,  2151, 16948, 11594,  1012,  1000,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
y_train[0]

0

In [ ]:
def compute_metrics(p):
  print(type(p))
  pred,labels = p
  pred = np.argmax(pred, axis=1)

  accuracy = accuracy_score(y_true=labels, y_pred= pred)
  recall = recall_score(labels, pred)
  precision = precision_score(labels, pred)
  f_score = f1_score(labels, pred)

  return {'accuracy':accuracy,
          'recall': recall,
          'precision':precision,
          'f1_score':f_score}


In [ ]:
# define trainer
args = TrainingArguments(
    output_dir = 'output',
    num_train_epochs = 1,
    per_device_train_batch_size = 8
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics
)

In [ ]:
# train
trainer.train()

C:\Users\nishant\anaconda3\envs\pytlearning\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.249900
1000,0.206600
1500,0.191300
2000,0.189100
2500,0.200100
3000,0.193000
3500,0.177000
4000,0.172100
4500,0.174000
5000,0.165200


TrainOutput(global_step=22355, training_loss=0.19055058465017832, metrics={'train_runtime': 9452.1804, 'train_samples_per_second': 18.92, 'train_steps_per_second': 2.365, 'total_flos': 4.705451802952704e+16, 'train_loss': 0.19055058465017832, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.15837477147579193,
 'eval_accuracy': 0.9558488033996869,
 'eval_recall': 0.7054009819967266,
 'eval_precision': 0.8086303939962477,
 'eval_f1_score': 0.7534965034965035,
 'eval_runtime': 738.9117,
 'eval_samples_per_second': 60.508,
 'eval_steps_per_second': 7.564,
 'epoch': 1.0}

In [ ]:
np.set_printoptions(True)

In [ ]:
# testing on real data
text = 'great work'
# text = 'stop insulting me'

inputs = tokenizer(text, padding=True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)

prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(prediction)

predictions = prediction.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.4490, -2.7836]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9947, 0.0053]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[1., 0.]], dtype=float32)

In [ ]:
trainer.save_model('MyModel')

In [ ]:
# from own finetuned model
model_2 = BertForSequenceClassification.from_pretrained('MyModel')
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# testing on real data
# text = 'great work'
text = 'what the shit'

inputs = tokenizer(text, padding=True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
print(outputs)

prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(prediction)

predictions = prediction.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-2.2026,  2.0674]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0138, 0.9862]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0., 1.]], dtype=float32)

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model_2.push_to_hub('fine-tuned-BertForSequenceClassification')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nish700/fine-tuned-BertForSequenceClassification/commit/cfc42ae0a287aa4c50e9abefd7492b8da186dfb5', commit_message='Upload BertForSequenceClassification', commit_description='', oid='cfc42ae0a287aa4c50e9abefd7492b8da186dfb5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('fine-tuned-BertForSequenceClassification')

CommitInfo(commit_url='https://huggingface.co/nish700/fine-tuned-BertForSequenceClassification/commit/2ad7570ea2cc2aa38d62450ffa88b204bfd4fdb3', commit_message='Upload tokenizer', commit_description='', oid='2ad7570ea2cc2aa38d62450ffa88b204bfd4fdb3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# X_test_tokenized.push_to_hub('fine-tuned-BertForSequenceClassification')

In [ ]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
tokenizer.push_to_hub('fine-tuned-BertForSequenceClassification')

CommitInfo(commit_url='https://huggingface.co/nish700/fine-tuned-BertForSequenceClassification/commit/6ad9d7916704bde191ab5f5e38b04037715beef6', commit_message='Upload tokenizer', commit_description='', oid='6ad9d7916704bde191ab5f5e38b04037715beef6', pr_url=None, pr_revision=None, pr_num=None)